In [1]:
import findspark
#初始化
findspark.init()
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql import SparkSession



In [127]:
# 定义数据库的地址，以及表，登录用户及密码
url = "jdbc:mysql://localhost:3306/sxzy"
table="df_gongzi"
#密码账户需要字典的形式传入
properties ={"user":"root","password":"521521"}
spark = SparkSession.builder.appName('My first app').getOrCreate()
df = spark.read.jdbc(url=url,table=table,properties=properties)
df.show(4)


+-------------------+------+----+-------------------+------+----+--------------------+--------------------------------+----------+
|               cxkh|  jymc|jdbz|               jysj|  jyje|jybz|             jydfzkh|                          jydfmc|       ymd|
+-------------------+------+----+-------------------+------+----+--------------------+--------------------------------+----------+
|0330059980130210812|  刘兵|  进|2016-02-21 15:30:27|6000.0| CNY|                null|                            null|2016-02-21|
|   4096662643338373|白翠平|  进|2016-07-06 15:14:08|1500.0| CNY|                null|                            null|2016-07-06|
|   4096662643338373|白翠平|  进|2016-07-13 16:48:09|2000.0| CNY|    6226223901739550|                          张晓梅|2016-07-13|
|6210810330000115469|  刘涛|  进|2016-01-20 09:36:43|6000.0| CNY|44201505900052512524|财付通支付科技有限公司客户备付金|2016-01-20|
+-------------------+------+----+-------------------+------+----+--------------------+------------------------------

In [4]:
df  # 查看字段，通过字段回去需要的数据

DataFrame[cxkh: string, jymc: string, jdbz: string, jysj: string, jyje: string, jybz: string, jydfzkh: string, jydfmc: string, ymd: string]

In [9]:
df.count(),len(df.columns)

(21136, 9)

In [10]:
df.printSchema()# 元数据分析，查看字段类型

root
 |-- cxkh: string (nullable = true)
 |-- jymc: string (nullable = true)
 |-- jdbz: string (nullable = true)
 |-- jysj: string (nullable = true)
 |-- jyje: string (nullable = true)
 |-- jybz: string (nullable = true)
 |-- jydfzkh: string (nullable = true)
 |-- jydfmc: string (nullable = true)
 |-- ymd: string (nullable = true)



In [8]:
df1=df.select(['cxkh','jymc']) # 按照列名选择

In [11]:
df1.show(5)

+-------------------+------+
|               cxkh|  jymc|
+-------------------+------+
|0330059980130210812|  刘兵|
|   4096662643338373|白翠平|
|   4096662643338373|白翠平|
|6210810330000115469|  刘涛|
|6210810330000115469|  刘涛|
+-------------------+------+
only showing top 5 rows



In [12]:
df2=df.filter((df['jyje']>100000) & (df['jysj']>'2015-01-01')) #按照条件进行筛选

In [14]:
df2

DataFrame[cxkh: string, jymc: string, jdbz: string, jysj: string, jyje: string, jybz: string, jydfzkh: string, jydfmc: string, ymd: string]

In [19]:
df3=df.select(['jyje'])
df3.show(3)

+------+
|  jyje|
+------+
|6000.0|
|1500.0|
|2000.0|
+------+
only showing top 3 rows



In [22]:
df3.printSchema()

root
 |-- jyje: string (nullable = true)



In [23]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

In [24]:
# 类型转换
df3 = df3.withColumn("jyje",df3.jyje.astype("int"))

In [25]:
# 生成密集性向量
assembler = VectorAssembler(inputCols=df3.columns,outputCol="features")
df_new=assembler.transform(df3)

In [27]:
df_new.show(2)

+----+--------+
|jyje|features|
+----+--------+
|6000|[6000.0]|
|1500|[1500.0]|
+----+--------+
only showing top 2 rows



In [28]:
Correlation.corr(df_new,'features').show()## pearson相关性

+-----------------+
|pearson(features)|
+-----------------+
|            1.0  |
+-----------------+



In [29]:
Correlation.corr(df_new,'features', 'spearman').show()# 计算spearman系数

+------------------+
|spearman(features)|
+------------------+
|             1.0  |
+------------------+



In [30]:
df.describe().show(５)

+-------+--------------------+------+-----+-------------------+------------------+-----+--------------------+----------------------------------+----------+
|summary|                cxkh|  jymc| jdbz|               jysj|              jyje| jybz|             jydfzkh|                            jydfmc|       ymd|
+-------+--------------------+------+-----+-------------------+------------------+-----+--------------------+----------------------------------+----------+
|  count|               21136| 21136|21136|              21136|             21136|21136|               17456|                             17345|     21136|
|   mean|1.393741088568533...|  null| null|               null| 158311.3010276308| null|1.038028163147666E24|                              null|      null|
| stddev|2.738862497578493...|  null| null|               null|1335809.6294491452| null|1.510245156186868...|                              null|      null|
|    min| 0330019980130136883|任志强|   进|2013-01-04 14:37:11|     

In [31]:
df3.describe().show()  # 描述性统计

+-------+------------------+
|summary|              jyje|
+-------+------------------+
|  count|             21136|
|   mean|158311.19081188494|
| stddev|1335809.6361363924|
|    min|              1000|
|    max|          61726798|
+-------+------------------+



In [32]:
df3.printSchema()#查看字段属性

root
 |-- jyje: integer (nullable = true)



In [33]:
df.filter(df['jymc']=='崔新平').show(2)  # 样本选择

+----------------+------+----+-------------------+---------+----+----------------+------+----------+
|            cxkh|  jymc|jdbz|               jysj|     jyje|jybz|         jydfzkh|jydfmc|       ymd|
+----------------+------+----+-------------------+---------+----+----------------+------+----------+
|6217680703321161|崔新平|  进|2015-12-03 11:58:24|5000000.0| CNY|6226980700701764|吴晓虎|2015-12-03|
|6217680703321161|崔新平|  进|2015-12-03 11:59:49|5000000.0| CNY|6226980700701764|吴晓虎|2015-12-03|
+----------------+------+----+-------------------+---------+----+----------------+------+----------+
only showing top 2 rows



In [34]:
df.filter(df['jymc']=='崔新平').select('cxkh','jdbz','jyje').show(2) # 样本+变量选择

+----------------+----+---------+
|            cxkh|jdbz|     jyje|
+----------------+----+---------+
|6217680703321161|  进|5000000.0|
|6217680703321161|  进|5000000.0|
+----------------+----+---------+
only showing top 2 rows



In [35]:
df.withColumn('test',(df['jyje']+100)).select('jyje','test').show(5) # 增加字段

+-------+-------+
|   jyje|   test|
+-------+-------+
| 6000.0| 6100.0|
| 1500.0| 1600.0|
| 2000.0| 2100.0|
| 6000.0| 6100.0|
|10000.0|10100.0|
+-------+-------+
only showing top 5 rows



In [39]:
# df.select('zysm').show()
ddd=df[df['jydfmc'].isNull()]  # 筛选未空

In [49]:
ddd1=ddd['jymc','jydfmc']

In [53]:
ddd1.fillna('wz').show(2) #缺失值填充
ddd1.na.fill('wz').show(2)  #缺失值填充

+------+------+
|  jymc|jydfmc|
+------+------+
|  刘兵|    wz|
|白翠平|    wz|
+------+------+
only showing top 2 rows

+------+------+
|  jymc|jydfmc|
+------+------+
|  刘兵|    wz|
|白翠平|    wz|
+------+------+
only showing top 2 rows



分组处理

In [55]:
df.groupBy('jymc').count().show(3)
df.groupBy('jymc').count().orderBy('count',ascending=False).show(3)

+------+-----+
|  jymc|count|
+------+-----+
|张丽丽|  453|
|高明明| 1075|
|吴晓虎|  233|
+------+-----+
only showing top 3 rows

+------+-----+
|  jymc|count|
+------+-----+
|  刘涛| 8141|
|张莉莉| 2598|
|高明明| 1075|
+------+-----+
only showing top 3 rows



In [61]:
df.groupBy('jyje').mean().show(5,False)

+-------+
|jyje   |
+-------+
|1664.34|
|1259.04|
|1488.74|
|1368.63|
|1857.14|
+-------+
only showing top 5 rows



In [82]:
df.groupBy('jymc','cxkh').agg({'jyje':'sum','jysj':'max'}).orderBy('sum(jyje)',ascending=False).show(5,False) #特定函数聚合求解

+------+-------------------+-------------------+--------------------+
|jymc  |cxkh               |max(jysj)          |sum(jyje)           |
+------+-------------------+-------------------+--------------------+
|崔新平|6229980707666660   |2019-11-22 15:32:39|7.1646981569E8      |
|崔新平|6217680705469992   |2019-10-23 10:23:24|2.4818253972000003E8|
|张丽丽|6229985520053194   |2019-03-01 16:10:24|2.0568639223999998E8|
|吴晓虎|6217680706701815   |2019-12-09 10:42:21|2.0276562496000004E8|
|杨晓龙|6222625430000169209|2019-12-14 10:48:07|1.8387798923999998E8|
+------+-------------------+-------------------+--------------------+
only showing top 5 rows



In [93]:
# df.select(['jymc','jydfmc']).stat.crosstab('jymc','jydfmc').show()
df.select(['cxkh','jymc']).dropDuplicates().show(5),
df.select(['cxkh','jymc']).dropDuplicates().count()

+-------------------+------+
|               cxkh|  jymc|
+-------------------+------+
|   6217685500292663|  刘涛|
|0509256802204209296|杨建进|
|   6216921002523698|杨建进|
|   6226223903387994|杨晓龙|
|       147970660363|郭凤兰|
+-------------------+------+
only showing top 5 rows



219

In [94]:
from pyspark.sql.functions import  udf
def jyje_level(x):
    if x< 100000 :
        return '一等'
    elif 100000<=x <500000:
        return '二等'
    else :
        return '三等'

In [124]:
from pyspark.sql.types import  StringType  #类似于ｌａｍｂｄａ
brand_udf=udf(jyje_level,StringType())  #注册ｕｄｆ
df3.withColumn('jyje_level',brand_udf(df3['jyje'])).show(5)   
brand_udf1=udf(lambda x: '+' if x>1000000 else '-',StringType())  #匿名函数
df3.withColumn('jyje_level1',brand_udf1(df3['jyje'])).show(5)


+-----+----------+
| jyje|jyje_level|
+-----+----------+
| 6000|      一等|
| 1500|      一等|
| 2000|      一等|
| 6000|      一等|
|10000|      一等|
+-----+----------+
only showing top 5 rows

+-----+-----------+
| jyje|jyje_level1|
+-----+-----------+
| 6000|          -|
| 1500|          -|
| 2000|          -|
| 6000|          -|
|10000|          -|
+-----+-----------+
only showing top 5 rows

